### От Ирины

**Расчет маневров при действительно опасном сближении.**

Если зафиксировано опасное сближение (ориентировочно до 100 м), то задача ИИ – определить параметры маневра для «защищаемого» объекта, чтобы он ушел от столкновения с целью минимизировать «потери». С точки зрения баллистики в «потери» я могу включить только затраты топлива. С точки зрения BigData в «потерях» можно учитывать много дополнительных данных – насколько много у аппарата останется топлива, помешает ли ему этот маневр выполнить какую-то запланированную целевую задачу (например, съемку определенного района) и т.д. Таким образом, задача ИИ – найти оптимальные (по величине этого функционала) параметры маневра, спасающего аппарат от опасного сближения. При этом дополнительно нужно проверять, что при этом маневре у «защищаемого» объекта не возникнет опасных сближений с какими-то другими посторонними объектами из базы.
В какой форме записать функционал – отдельная системная задача, где нужен мозговой штурм. Нужно ли сводить все разнообразные параметры в единую функцию с весовыми коэффициентами типа

$$
Ф=α1*ΔV+ α2*\text{флаг(не испортили маневром временной интервал выполнения съемки: 0 или 1)}+…+ αn*\text{[n-й критерий]}
$$

Либо все же распределить параметры на несколько функционалов, чтобы ИИ выдавал табличкой перечень найденных вариантов маневров с различными значениями по различным критериям, а окончательное решение принимает уже оператор или группа управления.


In [29]:
import numpy as np

def to_xyz(coordinates):
    """
    ф-я переводит координаты в xyz
    (пока случайно)
    """
    result = np.random.normal(size=[coordinates.shape[0], 3])
    return result
    
class API:
    
    def get_reward(self, state, next_state, current_reward):
        """
        state, next_state: dict where keys:
            'coordinates': dict where:
                {'st': np.array shape (n_satellites, 6)}:  satellites coordinates
                {'gb': np.array shape (n_items, 6)}:  garbage coordinates
            'fuel': float
            'trajectory_deviation_coef': float < 10
        current_reward: float
        ---
        output: float
        """
        # координаты КА
        satellites_coordinates = to_xyz(state['coordinates']['st'])
        # координаты остальных объектов
        items_state = to_xyz(state['coordinates']['gb'])

        # rewards
        # за сближение (просто считаем Евклидово расстояние между
        # КА и другим объектом) и экспоненту
        collision_danger = 0
        for j in range(satellites_coordinates.shape[0]):
            for i in range(items_state.shape[0]):
                # Евклидово расстояние
                item_danger = np.sum((satellites_coordinates[j] - items_state[i]) ** 2) ** 0.5
                # 
                item_danger = 1. / item_danger
                print 'item danger: ', item_danger
                collision_danger -= item_danger
        print 'collision danger: ', collision_danger
        
        # штраф за расход топлива  
        fuel_consumption = -(state['fuel'] - next_state['fuel'])
        print 'fuel consumption:', fuel_consumption
        
        # награда за движение близко к траектории
        traj_reward = -state['trajectory_deviation_coef']
        print 'traj reward:', traj_reward
        
        # whole reward    
        reward = (
            collision_danger
            + fuel_consumption
            + traj_reward
        ) 
        #^-- остальные rewards. можно с весом суммировать или в степени возволить
        
        new_reward = current_reward + reward
        print 'reward:', new_reward
        return new_reward


In [30]:
state = {
    'coordinates': {
        'st': np.random.normal(size=[2, 6]),
        'gb': np.random.normal(size=[20, 6])
    },
    'fuel': 10,
    'trajectory_deviation_coef': 10,
}
next_state = {
    'coordinates': {
        'st': np.random.normal(size=[2, 6]),
        'gb': np.random.normal(size=[20, 6])
    },
    'fuel': 8,
    'trajectory_deviation_coef': 8,
}
current_reward = 10

In [31]:
#проверка
A = API()
B = A.get_reward(state, next_state, current_reward)

item danger:  0.498466745071
item danger:  0.505707962511
item danger:  0.345736754166
item danger:  0.538586154379
item danger:  0.491386171993
item danger:  0.458367971405
item danger:  0.396947214784
item danger:  0.547117841118
item danger:  0.294715039125
item danger:  0.396861904869
item danger:  0.403445869308
item danger:  0.393570111174
item danger:  0.376394098979
item danger:  0.716307281698
item danger:  0.774957211384
item danger:  0.688528069775
item danger:  0.397535478441
item danger:  0.689358859187
item danger:  0.321259805633
item danger:  0.666872549284
item danger:  0.452366152774
item danger:  0.502126361898
item danger:  0.381115460318
item danger:  0.632207348333
item danger:  0.389563920871
item danger:  0.530615013332
item danger:  0.498194841765
item danger:  1.08947185479
item danger:  0.271015720709
item danger:  0.389370266896
item danger:  0.594968473303
item danger:  0.416797689912
item danger:  0.364747020107
item danger:  0.475987250756
item danger:  0

**вопросы**

1. проверить - reward за state или action? предыдущий state или следующий как второй брать
1. метод get_reward должен на входе еще иметь параметры - пороги всякие и прочее или все функциями с этими параметрами сделать
1. ф-я to_xyz
1. коэф отклонения от траектории
1. вместо времени жизни давайте введем коэф отклонения от траектории. Когда отклонение мало пусть награду получает.
1. заканчивать сессию когда кончается топливо

**потом**

* чтобы натренить модель можно сделать и натренить несколько "охотников", которые будут старатья сбить защищаемые объет. можно, например, разрешать им редко менять трактории, что они впрямую не летали за объектом, или скорость меньше задать.
* сделать нейронку (или что-то другое), которая будет говорить, какие объекты потенциально опасные. и потом можно типа там 100000 объектов назадавать, чтоб училась и смотрелось нормально. охотниками пусть случайные объекты выбираются через какое-то время
* штрафовать за большой угол

**визуализация**

* 2D
* подсвечивать маневры и отклонение от траектории
* подсвечивать охотников

**RL**

* encoder для state

**мысли**

можно reward:
+ за время без маневров
+ за время летения по правильной траектории
- за топливо
- за маневр
-!!! за столкновение

можно еще
+ за время без опасности
- за опасность
но, вроде, т.к. опасность => маневр, то штраф за маневр и время
без маневров это учитывают

ну или, кстати, можно еще за сближение <R штрафовать.